# A stats refresher notebook


In [75]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.stats as st

### Problem no. 1

You measure a value of 1.73 for a random variable with a mean of 2.20 and a standard deviation of 0.22. What is the z-value of the measurement?

In [76]:
(1.73 - 2.20) / 0.22

-2.136363636363637

### Problem no. 2

We have a sample of 100 voters and we know 56 would vote for candidate A and 44 would vote for candidate B. Assuming the sample is relevant, what is the probability that candidate B would win the elections? 

In [77]:
p_A = 56 / 100
p_B = (1-p_A)

# standard error of the population proportion
stderr = np.sqrt(p_A * p_B / 100.0) 
print(stderr)

### A < 50%
z = (0.5 - p_A) / stderr
print(z) # how many standard deviations from the prop proportion is

# what is the probability of < 0.5 givem 56/100 and our stderr
print(st.norm.cdf(0.5, loc=p_A, scale=stderr))

0.04963869458396342
-1.2087344460380716
0.11338244176241852


### Problem no. 3

What should be the number of voters that should declare they vote for candidate A for the candidate A to be certain that he will win the election? 

In [78]:
# p(A < 50%) = 5%. sample size is 100
z=st.norm.ppf(0.05)
#(0.5 - p_A) * sqrt(100) / sqrt((p_A - p_A*p_A)) = z
# 0.5 * sqrt(100) - sqrt(100) * p_A = z * sqrt(p_A - p_A * p_A) | ^2
# (0.5 * sqrt(100)) ^ 2 - 2 * 0.5 * sqrt(100) * sqrt(100) * p_A + 100 * p_A^2 = z^2 * (p_A - p_A^2)
# 25 - 100 * p_A + 100 * p_A^2 - z^2 * p_A + z^2 * p_A^2 = 0
# 25 - (100 + z^2) * p_A + (100 + z^2) * p_A^2 = 0
print(z**2)
# z = -1.64, z^2 = 2.7
# 25 - 102.7 * p_A + 102.7 * p_A^2 = 0
# x1,2 =(102.7 +- sqrt(102.7^2 - 4 * 102.7 * 25)) / (2 * 102.7)
# x1 = 0.42
# x2 = 0.58
# so p_A should be 0.58 => to have a confidence level of 95% that p_A > 50 there should be at least 0.58 * 100 = 58 votes

# test (ok to be slightly higher than 0.05 because I also rounded the z to 1.64):
print(st.norm.cdf(0.5, loc=0.58, scale=np.sqrt((0.58 * (1-0.58) / 100))))

2.705543454095415
0.052521495732662904


### Problem no. 4

In a similar analytical manner, we could also solve to find what would be the number of voters in the sample that would give us a confidence of 95% that candidate A would win, given that the sample proportion is 0.56. In this case the variable to search for would be N.

In [79]:
# the result of the cdf should be less than 0.05. The result is N=186
print(st.norm.cdf(0.5, loc=0.56, scale=np.sqrt((0.56 * (1-0.56) / 186))))

0.049625670476904964


### Problem no. 5

A study claims that teenagers check their phones, on average, 56 times per day. A random experiment on 100 teenagers shows that the teenagers in the sample check their approximatively 55 times, on average, per day, with a standard deviation of the sample of 11. Can the initial study be invalidated?

We will test two hypothesis:

H0 - 57 times per day
H11 - Not 57 times per day

and 

H12 - Less than 57 times per day

Speaking of hypothesis testing and when to choose one-sided test and when to choose a two-sided test, a very good [link](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-what-are-the-differences-between-one-tailed-and-two-tailed-tests/)

In [80]:
# first case, test H11 - double sided test
# we estimate the population stddev the same as the sample stddev. 
# we can use this because the sample size is relatively large, larger than 30 (an empirical value)
sem = 11 / np.sqrt(100)
print(sem)

p = st.norm.cdf(56, loc=55, scale=sem)
print(p)

print("Can H11 be accepted?", p > 0.975 or p < 0.025)
print("Can H12 be accepted?", p < 0.95)

1.1
0.818348929556551
Can H11 be accepted? False
Can H12 be accepted? True


## Problem no. 6

Same problem as above, but instead of 100 teenagers only 10 are interviewed. The results are [43, 33, 55, 57, 55, 68, 45, 63, 64, 30]. Can we invalidate the hypothesis that teenagers check their phones, on average, 57 times per day?

Since the sample size is small, we need to resort to the T-distribution.

In [81]:
arr = np.array([43, 33, 55, 57, 55, 68, 45, 63, 64, 30])
mean = np.mean(arr)
std = np.std(arr)

 # we are interested in the variation of the mean, not of the variation in the sample
sem = std/np.sqrt(len(arr))
print (mean, std, sem)

# df is degrees of freedom:
p = st.t.cdf(57, df=len(arr)-1, loc=mean, scale=sem)
print("Hypothesis can be rejected: ", p < 0.025 or p > 0.975, p)

51.3 12.38587905640936 3.9167588641630724
Hypothesis can be rejected:  False 0.9102172502388542


### Problem no. 7

A vaccine is tested by splitting the population in two - a part of the population is given a placebo, the other part the new vaccine. The question to answer is if the new vaccine has an effect or not. Let's assume that the for placebo we have 2023 patients and 54 got the disease. The new vaccine was administered to 2050 people and only 23 got the sickness afterwards. 

In [82]:
p1 = 54/2023
p2 = 23/2050
print("Proportions:", p1, p2)

variance_1 = p1 * (1-p1) / 2023
variance_2 = p2 * (1-p2) / 2050
print ("Variances:", variance_1, variance_2)

# distribution of differences between the two populations
mean_diff = p1-p2
variance_diff = variance_1 + variance_2

# SE of sample proportions
SE = np.sqrt(variance_diff)
print("Difference:", mean_diff, SE)

# 95% confidence interval for SE - conclusion, since the lower limit of the confidence interval > 0
# most likely the vaccine helps a bit
print("Confidence interval: ", mean_diff + SE * st.norm.ppf(0.025), mean_diff + SE * st.norm.ppf(0.975))

# But are the sample distributions of the sample proportions normal to start with?
print(p1 * 2023 > 10 and (1-p1) * 2023 > 10) # if n * p < 10 -> skewed to the right <=> median < mean
print(p2 * 2050 > 10 and (1-p2) * 2050 > 10) # if n * (1-p) < 10 -> skewed to the left <=> median > mean
# In our case, both are roughly normally distributed 

Proportions: 0.026693030153237766 0.01121951219512195
Variances: 1.2842566630981761e-05 5.4115291420612005e-06
Difference: 0.015473517958115815 0.0042724812197414
Confidence interval:  0.007099608642798908 0.02384742727343272
True
True


### Problem no. 8

Let's assume the usual seasonal birthrates for a hospital are as follows - winter 15%, spring - 25%, summmer - 30%, autumn - 30% out of an expected of 80 babies. In the first year of the pandemic the following rates have been registered - winter 10 babies, spring - 22 babies, summer - 28 babies, autumn - 40 babies. Was the 1st pandemic year an unsual year for this particular hospital? 

- *H0* - the first year of the pandemic was a normal year
- *H1* - the first year of the pandemic was an outstanding year

In [83]:
avg_babies_w = 0.15 * 80
avg_babies_sp = 0.25 * 80
avg_babies_su = 0.30 * 80
avg_babies_au = 0.30 * 80

print(avg_babies_w, avg_babies_sp, avg_babies_su, avg_babies_au)

chi_sq_w = (avg_babies_w - 10) ** 2 / avg_babies_w
chi_sq_sp = (avg_babies_sp - 22) ** 2 / avg_babies_sp
chi_sq_su = (avg_babies_su - 28) ** 2 / avg_babies_su
chi_sq_au = (avg_babies_au - 40) ** 2 /avg_babies_au

# the smaller the individual chi_sq is, the category fits to the average
# in our case, autumn does not fit at all, with the rest more or less inline
print(chi_sq_w, chi_sq_sp, chi_sq_su, chi_sq_au)

chi_sq = chi_sq_w + chi_sq_sp + chi_sq_su + chi_sq_au
print(chi_sq)

degrees_of_freedom = 4 - 1 # 4 seasons - 1

p = st.chi2.cdf(chi_sq, df=degrees_of_freedom)
# we can reject the null hypothesis that this was a normal year.
print("P-value:", p, ", can we reject:", p < 0.025 or p > 0.975) 

# and directly with python function - same returns
st.chisquare([10, 22, 28, 40], f_exp=[avg_babies_w, avg_babies_sp, avg_babies_su, avg_babies_au])

12.0 20.0 24.0 24.0
0.3333333333333333 0.2 0.6666666666666666 10.666666666666666
11.866666666666665
P-value: 0.9921458560317754 , can we reject: True


Power_divergenceResult(statistic=11.866666666666665, pvalue=0.007854143968224583)

### Problem no. 9

A company has 4 teams, each led by a different manager, each with 4 people. Let's assume the managers are A, B, C and D. The manager ratings for each individual team as as follows:

- *A* - 5, 5, 3, 3,
- *B* - 8, 5, 4, 3,
- *C* - 8, 6, 5, 5,
- *D* - 8, 6, 4, 2,

We want to test:

- *H0* - all managers are roughly the same (same mean)
- *H1* - not all managers are the same (groups have different mean)

In [84]:
scores = np.array([
    [5, 5, 3, 3],
    [8, 5, 4, 3],
    [8, 6, 5, 5],
    [8, 6, 4, 2]])

means = np.mean(scores, axis=1)
grand_mean = np.mean(means)
#print(means, grand_mean)

# total sum of squares (for each, with the grand_mean)
SST = np.sum(np.sum((scores - grand_mean) ** 2, axis=1))
#print(SST)

# sum of squares within (for each, with the group mean)
SSW = np.sum(np.sum([(scores[i] - means[i]) ** 2 for i in range(0, 4)], axis=1))
#print(SSW)

# sum of squares between
SSB = SST - SSW

# f_statistic - comparing the variance between groups with the variance within groups
m = 4 # number of managers (groups) 
nt = scores.size # the total number of observations 

dfn = m-1 # degrees of freedom within number of groups
dfd = nt-m # degrees of freedom within the total dataset

f_statistic = (SSB / dfn) / (SSW / dfd)
print("f-statistic:", f_statistic, "p-value:", 1 - st.f.cdf(f_statistic, dfn=dfn, dfd=dfd))

# and directly with the scipy
st.f_oneway(scores[0], scores[1], scores[2], scores[3])

# in our case, there is no significant level of difference bwtween these 4 managers

f-statistic: 0.7272727272727273 p-value: 0.5551086637384909


F_onewayResult(statistic=0.7272727272727273, pvalue=0.5551086637384908)